In [ ]:
import os
import time
import pandas as pd

## Tracking the progress of the feature extraction process

In [1]:
df_path = os.path.join(os.getcwd(), 'data', 'tweets.csv')
df = pd.read_csv(df_path)
df = df.drop_duplicates(subset=['tw_texto'], keep='first')

processed_df_path = os.path.join(os.getcwd(), 'data', 'tweets_gpt_features.csv')
processed_df = pd.read_csv(processed_df_path)
processed_df = processed_df.drop_duplicates(subset=['tw_texto'], keep='first')

print(
    f"Time: {time.strftime('%H:%M:%S')}",
    f"Total tweets: {len(df)}",
    f"Processed tweets: {len(processed_df)}",
    f"Remaining tweets: {len(df) - len(processed_df)}",
    f"Percentage of processed tweets: {len(processed_df) / len(df) * 100:.2f}%",
    sep='\n'
)

Time: 09:58:28
Total tweets: 1454
Processed tweets: 1454
Remaining tweets: 0
Percentage of processed tweets: 100.00%


## Merge extracted features into original the dataset fetched from Twitter

In [29]:
processed_df = processed_df.iloc[:, :10].dropna(axis=0)
final_df = pd.merge(df, processed_df, on='tw_texto', how='left').dropna()
final_df.head()

,tw_tweet,tw_usuario,tw_fecha,tw_texto,tw_sensitivo,tw_retweets,tw_replies,tw_likes,tw_quotes,tw_impresiones,...,candidato,tw_candidate,tw_valencia,tw_emocion,tw_postura,tw_tono,tw_amabilidad,tw_legibilidad,tw_controversialidad,tw_informatividad
0,1658254861371056135,334344110,2023-05-15,La Sala Sexta del Tribunal de lo Contencioso A...,False,4,1,3,0,1030,...,carlos pineda,carlos pineda,negativo,preocupación,desaprobación,informativo,0.8,0.9,0.5,0.9
1,1658254410328293382,1444181526506315783,2023-05-15,Sala requiere informe circunstanciado al Regis...,False,2,0,4,1,1915,...,carlos pineda,carlos pineda,negativo,enojo,desaprobación,informativo,0.5,1.0,0.7,0.9
3,1658252064424615937,2343910962,2023-05-15,#EleccionesGuate2023 🇬🇹 | Sala da trámite a am...,False,5,4,24,3,5701,...,carlos pineda,carlos pineda,neutro,otro,indiferencia,informativo,0.8,1.0,0.5,0.9
4,1658251948041162752,56183126,2023-05-15,#BreakingNews Sala requiere informe circunstan...,False,10,3,23,3,7147,...,carlos pineda,carlos pineda,negativo,enojo,desaprobación,informativo,0.5,1.0,0.8,1.0
5,1658249936050200577,4025829620,2023-05-15,No soy partidario ni de cerca de Carlos Pineda...,False,0,0,2,1,303,...,carlos pineda,carlos pineda,neutro,indiferencia,esperanza,serio,0.8,0.9,0.4,0.8


## Cleanse the data and write the final dataset

In [28]:
final_df.columns

Index(['tw_tweet', 'tw_usuario', 'tw_fecha', 'tw_texto', 'tw_sensitivo',
       'tw_retweets', 'tw_replies', 'tw_likes', 'tw_quotes', 'tw_impresiones',
       'tw_idioma', 'candidato', 'tw_candidate', 'tw_valencia', 'tw_emocion',
       'tw_postura', 'tw_tono', 'tw_amabilidad', 'tw_legibilidad',
       'tw_controversialidad', 'tw_informatividad'],
      dtype='object')

In [31]:
final_df = final_df[[
    'tw_fecha', 'tw_texto', 'tw_candidate', 
    'tw_retweets', 'tw_replies', 'tw_likes', 
    'tw_quotes', 'tw_impresiones', 'tw_valencia', 
    'tw_emocion', 'tw_postura', 'tw_tono', 
    'tw_amabilidad', 'tw_legibilidad', 'tw_controversialidad', 
    'tw_informatividad', 'tw_sensitivo'
]]

final_df.head()

,tw_fecha,tw_texto,tw_candidate,tw_retweets,tw_replies,tw_likes,tw_quotes,tw_impresiones,tw_valencia,tw_emocion,tw_postura,tw_tono,tw_amabilidad,tw_legibilidad,tw_controversialidad,tw_informatividad,tw_sensitivo
0,2023-05-15,La Sala Sexta del Tribunal de lo Contencioso A...,carlos pineda,4,1,3,0,1030,negativo,preocupación,desaprobación,informativo,0.8,0.9,0.5,0.9,False
1,2023-05-15,Sala requiere informe circunstanciado al Regis...,carlos pineda,2,0,4,1,1915,negativo,enojo,desaprobación,informativo,0.5,1.0,0.7,0.9,False
3,2023-05-15,#EleccionesGuate2023 🇬🇹 | Sala da trámite a am...,carlos pineda,5,4,24,3,5701,neutro,otro,indiferencia,informativo,0.8,1.0,0.5,0.9,False
4,2023-05-15,#BreakingNews Sala requiere informe circunstan...,carlos pineda,10,3,23,3,7147,negativo,enojo,desaprobación,informativo,0.5,1.0,0.8,1.0,False
5,2023-05-15,No soy partidario ni de cerca de Carlos Pineda...,carlos pineda,0,0,2,1,303,neutro,indiferencia,esperanza,serio,0.8,0.9,0.4,0.8,False


In [36]:
final_df.columns = [col.replace('tw_', '') for col in final_df.columns]

final_df = (
    final_df
    .rename(columns={
        'texto': 'tweet',
        'candidate': 'candidato'
    })
)

In [38]:
final_df.head()

,fecha,tweet,candidato,retweets,replies,likes,quotes,impresiones,valencia,emocion,postura,tono,amabilidad,legibilidad,controversialidad,informatividad,sensitivo
0,2023-05-15,La Sala Sexta del Tribunal de lo Contencioso A...,carlos pineda,4,1,3,0,1030,negativo,preocupación,desaprobación,informativo,0.8,0.9,0.5,0.9,False
1,2023-05-15,Sala requiere informe circunstanciado al Regis...,carlos pineda,2,0,4,1,1915,negativo,enojo,desaprobación,informativo,0.5,1.0,0.7,0.9,False
3,2023-05-15,#EleccionesGuate2023 🇬🇹 | Sala da trámite a am...,carlos pineda,5,4,24,3,5701,neutro,otro,indiferencia,informativo,0.8,1.0,0.5,0.9,False
4,2023-05-15,#BreakingNews Sala requiere informe circunstan...,carlos pineda,10,3,23,3,7147,negativo,enojo,desaprobación,informativo,0.5,1.0,0.8,1.0,False
5,2023-05-15,No soy partidario ni de cerca de Carlos Pineda...,carlos pineda,0,0,2,1,303,neutro,indiferencia,esperanza,serio,0.8,0.9,0.4,0.8,False


In [39]:
final_df.to_csv(os.path.join(os.getcwd(), 'data', 'tweets_gpt.csv'), index=False)